# Introduction to the PIC-SURE API
This is a tutorial notebook aimed to get the user quickly up and running with the PIC-SURE API. 

## PIC-SURE python API
### What is PIC-SURE?
As part of the BioData Catalyst Initiative, the Patient Information Commons: Standard Unification of Research Elements (PIC-SURE) platform has been integrating clinical and genomic datasets from multiple TOPMed and TOPMed-related studies funded by the National Heart, Lung, and Blood Institute (NHLBI). 

Original data exposed through the PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and esposes the different study datasets in a single tabular format. By simplifying the process of data extraction, it allows investigators to focus on downstream analysis and to facilitate reproducible science. 

### More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the database the same way using either language.

PIC-SURE is a larger project from which the R and python PIC-SURE APIs are only a small part. Among other things, PIC-SURE also offers a graphical user interface that allows researchers to explore variables across multiple studies, filter participants that match criteria, and create cohorts from this interactive exploration.

The python API is actively developed by the Avillach Lab at Harvard Medical School. 

PIC-SURE API GitHub repositories:
* https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-client

 ------- 

## Getting your user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

## Environment set-up

### Pre-requisites
**Check with Jason about this**
* python 3.6 or later
* pip python package manager, already available in most systems with a python interpreter installed (link to pip)

### Install packages
The first step to using the PIC-SURE API is to install the packages needed. The following code installs the PIC-SURE API components from GitHub, specifically:
* PIC-SURE Client
* PIC-SURE Adapter

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git@new-search

import PicSureBdcAdapter

## Connecting to a PIC-SURE resource

The following is required to get access to the PIC-SURE API:
* a network URL
* a user-specific security token

The following code specifies the network URL as the BioData Catalyst Powered by PIC-SURE URL and references the user-specific token saved as `token.txt`.

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token) file.

In [ ]:
# Uncomment production URL when testing in production
# PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
token_file = "token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)

## Getting help with the PIC-SURE API
Each of the objects in the PIC-SURE API have a `help()` method that you can use to get more information about its functionalities.

In [ ]:
bdc.help()

For example, the above output lists and briefly defines the four methods that can be used with the `bdc` resource. 

## Using the PIC-SURE variables dictionary
Now that you have set up your connection to the PIC-SURE API, let's determine which study or studies you are authorized to access. The `dictionary` method can be used to search the data dictionary for a specific term or to retrieve information about all the variables you are authorized to access.

In [ ]:
dictionary = bdc.useDictionary().dictionary() # Set up the dictionary
all_variables = dictionary.find() # Retrieve all variables you have access to

In [ ]:
## Code used to identify a bug

concepts = all_variables.listPaths()
for i in concepts:
    phs = i.split("\\")[1]
    if 'phs' not in phs:
        print(i)
# Note: all concept paths start with phs
# There aren't _consents or anything like that
print(all_variables.count())
print(len(concepts))

In [ ]:
all_variables.listPaths()

*Note: if you do not see any variables, you are not authorized to access any data. The rest of the notebook will not work.* 

The above output lists all of the variables you are authorized to access in PIC-SURE. These variables are organized in the "phs", "pht", and "phv" format that is used by the [database of Genotypes and Phenotypes (dbGaP)](https://www.ncbi.nlm.nih.gov/gap/). The concept path is organized in the following format:

**\\\phs\\\pht\\\phv\\\variable name\\\**

* **phs** corresponds to the study accession number 
* **pht** corresponds to the trait table accession number
* **phv** corresponds to the variable accession number
* the **variable name** corresponds to the encoded variable name that was used by the submitters of the data

Generally, a given study will have several tables, and those tables have several variables. You can find more information on the dbGaP data structure [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2031016/).

We can now use the `dataframe()` method to determine which studies you are authorized to access. 

In [ ]:
all_variables.dataframe().head()

This view shows the data dictionary results in a readable dataframe format. Let's save a phs number from the `studyId` to use for the rest of the notebook.

In [ ]:
phs_number = all_variables.dataframe()['studyId'].unique()[0]
print("The phs accession being used is", phs_number)

In [ ]:
openPicSure = bdc.useOpenPicSure()
authPicSure = bdc.useAuthPicSure()